# Aula 04 - Limpeza de dados

## 4.1. Objetivos de aprendizagem

### 4.1.1. Básico

1. Entender o conceito de limpeza de dados
2. Utilizar os 4 principais verbos para limpeza dos dados (`tidyr`)
    * `gather()`
    * `separate()`
    * `spread()`
    * `unite()`
3. Cascatear a execução de funções utilizando `pipes`

### 4.1.2. Intermediário

4. Utilizar `sep`, `extra` e `convert` para tratar dados cuja limpeza seja um pouco mais complexa

### 4.1.3. Avançado

5. Utilizar expressões regulares para separar colunas complexas

## 4.2. Recursos

* [Capítulo 12: Limpeza de dados](https://r4ds.had.co.nz/tidy-data.html) do livro _R for Data Science_
* [Capítulo 13: Pipes](https://r4ds.had.co.nz/pipes.html) do livro _R for Data Science_
* [Transformação de dados: _cheat sheet_](https://rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf)

## 4.3. Ajustes

In [1]:
home <- path.expand("~")
lib_dir <- file.path(file.path(home, "R"), "lib")
dir.create(lib_dir, showWarnings = FALSE)

library(utils)
.libPaths(c(lib_dir, .libPaths()))

# libraries needed
library(tidyverse)
library(readxl)

set.seed(8675309) # makes sure random numbers are reproducible

Warning message:
"package 'tidyverse' was built under R version 3.6.2"


ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'rlang' 0.3.4 is already loaded, but >= 0.4.0 is required


## 4.4. Três regras para a limpeza de dados

* Cada variável deve tersua própria coluna
* Cada observação deve ter sua própria linha
* Cada valor deve ter sua própria célula

Essa tabela tem três observações por linha e a coluna `total_meanRT` possui dois valores.


id | score_1 | score_2  | score_3 | rt_1 | rt_2 | rt_3 | total_meanRT
---|---------|----------|---------|------|------|------|--------------
1 | 2 | 5 | 6 | 939 | 844 | 788 | 13 (857)
2 | 3 | 6 | 4 | 718 | 659 | 764 | 13 (714)
3 | 4 | 7 | 3 | 857 | 890 | 859 | 14 (869)
4 | 5 | 4 | 2 | 902 | 900 | 959 | 11 (920)
5 | 1 | 2 | 5 | 757 | 823 | 901 | 8 (827)

Essa á a versão após a limpeza

id | trial | rt | score | total | mean_rt
---|-------|----|-------|-------|---------
1 | 1 | 939 | 2 | 13 | 857
1 | 2 | 844 |	5 |	13 |	857
1 |	3 |	788 |	6 |	13 |	857
2 |	1 |	718 |	3 |	13 |	714
2 |	2 |	659 |	6 |	13 |	714
2 |	3 |	764 |	4 |	13 |	714
3 |	1 |	857 |	4 |	14 |	869
3 |	2 |	890 |	7 |	14 |	869
3 |	3 |	859 |	3 |	14 |	869
4 |	1 |	902 |	5 |	11 |	920
4 |	2 |	900 |	4 |	11 |	920
4 |	3 |	959 |	2 |	11 |	920
5 |	1 |	757 |	1 |	8  | 827
5 |	2 |	823 |	2 |	8  | 827
5 |	3 |	901 |	5 |	8  | 827

## 4.5. Limpando os dados

Baixe o dataset [personality.csv](https://psyteachr.github.io/msc-data-skills/data/personality.csv). Os dados pertencem a um questionário de personalidade de 5-fatores (OCEAN). Cada questão está rotulada de acordo com o domínio (Op = openness, Co = concientiousness, Ex = extraversion, Ag = agreeableness, and Ne = neuroticism) e o número da questão.


In [4]:
ocean <- read_csv("https://psyteachr.github.io/msc-data-skills/data/personality.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  date = col_date(format = "")
)

See spec(...) for full column specifications.



Utilizamos alguma das funções já conhecidas para ter uma ideia do conjunto de dados

In [7]:
ocean

user_id,date,Op1,Ne1,Ne2,Op2,Ex1,Ex2,Co1,Co2,⋯,Ex7,Ne7,Co9,Op7,Ne8,Ag6,Ag7,Co10,Ex8,Ex9
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2006-03-23,3,4,0,6,3,3,3,3,⋯,3,NA,3,0,2,NA,3,1,2,4
1,2006-02-08,6,0,6,0,0,0,0,0,⋯,6,0,6,6,0,6,0,6,0,6
2,2005-10-24,6,0,6,0,0,0,0,0,⋯,5,1,5,5,1,5,1,5,1,5
5,2005-12-07,6,4,4,4,2,3,3,3,⋯,4,2,4,5,1,2,1,5,4,5
8,2006-07-27,6,1,2,6,2,3,5,4,⋯,1,0,3,5,1,3,1,3,3,5
108,2006-02-28,3,2,1,4,4,4,4,3,⋯,2,2,4,4,1,4,3,5,4,2
233,2006-11-08,3,3,2,4,4,5,3,3,⋯,5,4,5,6,5,5,3,1,2,3
298,2005-11-08,6,4,3,0,3,2,4,4,⋯,3,4,3,2,4,6,5,2,4,3
426,2006-09-05,6,0,1,0,5,5,5,5,⋯,6,0,5,1,0,1,0,5,6,6


In [8]:
glimpse(ocean)

Observations: 15,000
Variables: 43
$ user_id <dbl> 0, 1, 2, 5, 8, 108, 233, 298, 426, 436, 685, 807, 871, 881, 9…
$ date    <date> 2006-03-23, 2006-02-08, 2005-10-24, 2005-12-07, 2006-07-27, …
$ Op1     <dbl> 3, 6, 6, 6, 6, 3, 3, 6, 6, 3, 4, 5, 5, 5, 6, 4, 1, 2, 5, 6, 4…
$ Ne1     <dbl> 4, 0, 0, 4, 1, 2, 3, 4, 0, 3, 3, 3, 2, 1, 1, 3, 4, 5, 2, 4, 5…
$ Ne2     <dbl> 0, 6, 6, 4, 2, 1, 2, 3, 1, 2, 5, 5, 3, 1, 1, 1, 1, 6, 1, 2, 5…
$ Op2     <dbl> 6, 0, 0, 4, 6, 4, 4, 0, 0, 3, 4, 3, 3, 4, 5, 3, 3, 4, 1, 6, 6…
$ Ex1     <dbl> 3, 0, 0, 2, 2, 4, 4, 3, 5, 4, 1, 1, 3, 3, 1, 3, 5, 1, 0, 4, 1…
$ Ex2     <dbl> 3, 0, 0, 3, 3, 4, 5, 2, 5, 3, 4, 1, 3, 2, 1, 6, 5, 3, 4, 4, 1…
$ Co1     <dbl> 3, 0, 0, 3, 5, 4, 3, 4, 5, 3, 3, 3, 1, 5, 5, 4, 4, 5, 6, 4, 2…
$ Co2     <dbl> 3, 0, 0, 3, 4, 3, 3, 4, 5, 3, 5, 3, 3, 4, 5, 1, 5, 4, 5, 2, 5…
$ Ne3     <dbl> 0, 0, 0, 1, 0, 1, 4, 4, 0, 4, 2, 5, 1, 2, 5, 5, 2, 2, 1, 2, 5…
$ Ag1     <dbl> 2, 0, 0, 4, 6, 5, 5, 4, 2, 5, 4, 3, 2, 4, 5, 3, 5, 5, 5, 4, 4…
$ Ag2     <dbl> 1

### 4.5.1. gather()

`gather(data, key = "key", value = "value", ..., na.rm = FALSE, convert = FALSE, factor_key = FALSE)`

* `key` é a o nome da nova coluna que armazenará os headers dos dados agrupados; no exemplo se chama "_question_"
* `value` é o nome da variável que armazernará os valores dos dados agrupados; no exemplo se chama "_score_"
* `...` se referem às colunas que serão agrupadas. É possível utilizar o nome para chamar as colunas. Ex.: `col1, col2, col3, col4`, ou `col1:col4`, ou pelos seus números (`8, 9, 10` ou `8:10`)
* `na.rm` determina se as linhas com valor `NA` devem ser removidas
* `convert` se for necessário converter automaticamente os valores para outro tipo
* `factor_key` se os valores devem ser armazenados como um _factor_ (com a mesma ordem da tabela) ou como um vetor de caracteres

`ocean` está no formato largo ( _wide_ ), com uma coluna separada para cada pergunta. Mude para o formato longo, com uma linha para cada observação de usuário/pergunta. O dataframe resultante deve ter as colunas: `user_id`, `date`, `question` e `score`.

In [10]:
ocean_gathered <- gather(ocean, "question", "score", Op1:Ex9)
ocean_gathered

user_id,date,question,score
<dbl>,<date>,<chr>,<dbl>
0,2006-03-23,Op1,3
1,2006-02-08,Op1,6
2,2005-10-24,Op1,6
5,2005-12-07,Op1,6
8,2006-07-27,Op1,6
108,2006-02-28,Op1,3
233,2006-11-08,Op1,3
298,2005-11-08,Op1,6
426,2006-09-05,Op1,6


### 4.5.2. separate()

`separate(data, col, into, sep = "[^[:alnum:]]+", remove = TRUE, convert = FALSE, extra = "warn", fill = "warn")`

* `col` é a coluna que será tratada em separado
* `into` é um vetor com o nome das novas colunas
* `sep` é o caracter que será utilizado para separar as novas colunas. Podem ser utilizados quaisquer caracteres quenão sejam alfanuméricos, tais como `.,_-/:`
* `remove` determina se a coluna que será tratada em separado (`col`) deve ser removida da nova tabela de dados
* `convert` se for necessário converter automaticamente os valores para outro tipo
* `extra` controla o que acontece quando houver muitas ocorrências
* `fill` controla o que acontece quando não houver ocorrências suficientes

Vamos dividir a pergunta em duas colunas: `domain` e `qnumber`. Não existe um caracter específico para realizar a divisão dos dados, mas é possível separar uma coluna depois deuma certa quantidade de caracteres, utilizando um parâmetro inteiro na função `sep`.Para separar a string "abcde" depois do terceiro caractere, por exemplo, utilize `sep = 3`, resultando em `c("abc", "de")`. 

Também é possível utilizar um número negativo para separar os dados contando os caracteres da direita para a esquerda. Para separar os números das palavras "lisa03" e "amanda38" utilize `sep = -2`.

In [12]:
ocean_sep <- separate(ocean_gathered, question, c("domain", "qnumber"), sep = 2)
ocean_sep

user_id,date,domain,qnumber,score
<dbl>,<date>,<chr>,<chr>,<dbl>
0,2006-03-23,Op,1,3
1,2006-02-08,Op,1,6
2,2005-10-24,Op,1,6
5,2005-12-07,Op,1,6
8,2006-07-27,Op,1,6
108,2006-02-28,Op,1,3
233,2006-11-08,Op,1,3
298,2005-11-08,Op,1,6
426,2006-09-05,Op,1,6


### 4.5.3. unite()

`unite(data, col, ..., sep = "_", remove = TRUE)`

* `col` é a nova coluna com os dados reunidos
* `...` se refere às colunas que serão reunidas
* `sep` é o caractere que será utilizado para separar as colunas que foram reunidas
* `remove` determina se as colunas que foram reunidas (`...`) serão removidas da nova tabela de dados. O padrão é removê-las

Vamos reunir novamente as colunas `domain` e `qnumber` em uma nova coluna chamada `domain_n`. Para reuni-las utilizamos o formato `Op_Q1`.

In [13]:
ocean_unite <- unite(ocean_sep, "domain_n", domain, qnumber, sep = "_Q")
ocean_unite

user_id,date,domain_n,score
<dbl>,<date>,<chr>,<dbl>
0,2006-03-23,Op_Q1,3
1,2006-02-08,Op_Q1,6
2,2005-10-24,Op_Q1,6
5,2005-12-07,Op_Q1,6
8,2006-07-27,Op_Q1,6
108,2006-02-28,Op_Q1,3
233,2006-11-08,Op_Q1,3
298,2005-11-08,Op_Q1,6
426,2006-09-05,Op_Q1,6


### 4.5.4. spread()

`spread(data, key, value, fill = NA, convert = FALSE, drop = TRUE, sep = NULL)`

Também é possível reverter o processo de junção da função `unite()`. Basta converter os dados do formato longo ( _long_ ) para o formato largo ( _wide_ ). 

* `key` é a o nome da nova coluna que armazenará os novos headers
* `value` é o nome da coluna que contém os valores que passarão pela operação de `spread()`

In [14]:
ocean_spread <- spread(ocean_unite, domain_n, score)
ocean_spread

user_id,date,Ag_Q1,Ag_Q2,Ag_Q3,Ag_Q4,Ag_Q5,Ag_Q6,Ag_Q7,Co_Q1,⋯,Ne_Q6,Ne_Q7,Ne_Q8,Op_Q1,Op_Q2,Op_Q3,Op_Q4,Op_Q5,Op_Q6,Op_Q7
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2006-03-23,2,1,1,1,3,NA,3,3,⋯,1,NA,2,3,6,2,3,6,0,0
1,2006-02-08,0,6,0,0,6,6,0,0,⋯,6,0,0,6,0,6,0,6,6,6
2,2005-10-24,0,6,1,1,5,5,1,0,⋯,5,1,1,6,0,5,1,5,5,5
5,2005-12-07,4,0,1,4,0,2,1,3,⋯,1,2,1,6,4,5,6,2,1,5
8,2006-07-27,6,5,0,6,2,3,1,5,⋯,0,0,1,6,6,5,6,5,6,5
108,2006-02-28,5,4,4,5,5,4,3,4,⋯,1,2,1,3,4,4,3,4,4,4
233,2006-11-08,5,5,4,5,6,5,3,3,⋯,3,4,5,3,4,3,3,3,6,6
298,2005-11-08,4,3,4,6,2,6,5,4,⋯,4,4,4,6,0,2,0,2,0,2
426,2006-09-05,2,4,3,6,2,1,0,5,⋯,0,0,0,6,0,4,6,6,0,1


## 4.6. Pipes

Pipes são uma forma de organizar o código num formato legível. Digamos que exista uma pequena tabela com com o ID de 10 participantes, duas colunas com variáveis do tipo A e duas colunas com variáveis do tipo B. Queremos calcular a média das variáveis A, a média das variáveis  B e retornar uma tabela com 10 linhas (uma para cada participante) e 3 colunas (`id`, `A_mean` e `B_mean`).

Uma forma de resolver o problema é criar um novo objeto a cada passo e utilizar o mesmo objeto no próximo passo. É muito fácil implementar essa abordagem, mas foram criados 6 objetos desnecessários no ambiente. Na criação de scripts maiores esse desperdícios de recursos pode ser problemático.

In [15]:
# make a data table with 10 subjects
data_original <- tibble(
  id = 1:10,
  A1 = rnorm(10, 0),
  A2 = rnorm(10, 1),
  B1 = rnorm(10, 2),
  B2 = rnorm(10, 3)
)

# gather columns A1 to B2 into "variable" and "value" columns
data_gathered <- gather(data_original, variable, value, A1:B2)

# separate the variable column at the _ into "var" and "var_n" columns
data_separated <- separate(data_gathered, variable, c("var", "var_n"), sep = 1)

# group the data by id and var
data_grouped <- group_by(data_separated, id, var)

# calculate the mean value for each id/var 
data_summarised <- summarise(data_grouped, mean = mean(value))

# spread the mean column into A and B columns
data_spread <- spread(data_summarised, var, mean)

# rename A and B to A_mean and B_mean
data <- rename(data_spread, A_mean = A, B_mean = B)

data

id,A_mean,B_mean
<int>,<dbl>,<dbl>
1,-0.7730674,3.681005
2,1.3722310,3.098768
3,0.2664372,3.085310
4,1.1847138,2.557764
5,0.5354135,1.985066
6,1.9798392,2.727162
7,0.2928261,2.877059
8,0.3861176,2.610009
9,0.7107392,2.282698


Uma forma de evitar a criação de objetos extras é aninhar as funções, substituindo literalmente cada objeto de dados com o código utilizado para gerá-lo no passo anterior. Para pequenos conjuntos de funções, a abordagem pode ser bastante útil.

In [17]:
mean_petal_width <- round(mean(iris$Petal.Width), 2)
mean_petal_width

[1] 1.2

Contudo, a abordagem pode ser bastante confusa para longas cadeias de função

In [18]:
# do not ever do this!!
data <- rename(
  spread(
    summarise(
      group_by(
        separate(
          gather(
            tibble(
              id = 1:10,
              A1 = rnorm(10, 0),
              A2 = rnorm(10, 1),
              B1 = rnorm(10, 2),
              B2 = rnorm(10,3)), 
            variable, value, A1:B2), 
          variable, c("var", "var_n"), sep = 1), 
        id, var), 
      mean = mean(value)), 
    var, mean), 
  A_mean = A, B_mean = B)

data

id,A_mean,B_mean
<int>,<dbl>,<dbl>
1,0.2498205,3.1245687
2,0.5568939,3.4324043
3,0.1100131,2.3099954
4,-0.2255821,0.6138334
5,0.4810830,1.9680601
6,-0.0490533,3.6847839
7,-0.2632859,2.2093581
8,-0.1240991,1.6153072
9,1.1437207,2.6694551


A utilização de _pipes_ permite o envio do resultado de cada função para a próxima, permitindo organizar logicamente o código sem criar muitos objetos extras.

In [21]:
# calculate mean of A and B variables for each participant
data <- tibble(
  id = 1:10,
  A1 = rnorm(10, 0),
  A2 = rnorm(10, 1),
  B1 = rnorm(10, 2),
  B2 = rnorm(10,3)
) %>%
  gather(variable, value, A1:B2) %>%
  separate(variable, c("var", "var_n"), sep=1) %>%
  group_by(id, var) %>%
  summarise(mean = mean(value)) %>%
  spread(var, mean) %>%
  rename(A_mean = A, B_mean = B)

data

id,A_mean,B_mean
<int>,<dbl>,<dbl>
1,-0.09854705,3.577243
2,1.17001726,2.335873
3,0.63541259,2.656009
4,0.47562347,1.920438
5,-0.05311997,2.761962
6,0.08253345,3.003791
7,-0.28984292,1.673570
8,0.92170142,2.314281
9,-0.49681789,1.829271


O código pode ser lido no seguinte conjunto e passos:

1. Crie um tibble chamado `data` com:
    * ID de 1 a 10;
    * A1 de 10 números aleatórios oriundos de uma distribuição normal;
    * A2 de 10 números aleatórios oriundos de uma distribuição normal;
    * B1 de 10 números aleatórios oriundos de uma distribuição normal;
    * B2 de 10 números aleatórios oriundos de uma distribuição normal.
2. Utilize `gather()` para criar as as colunas `variable` e `value` a partir das colunas `A_1` a `B_2`;
3. Separe a coluna `variable` em duas novas colunas chamadas `var` e `var_n` separados no caractere 1;
4. Agrupe pelas colunas `id` e `var`;
5. Aplica a função `summarise()` criando a coluna `mean` como a média de `value` para cada grupo;
6. Utilize `spread()` para criar novas colunas com o nome das chaves em `var` e os valores em `mean`;
7. Renomeie as colunas `A` e `B` para criar `A_mean` e `B_mean`.

Quando o código estiver ficando grande demais ou quando for necessário debugar alguma parte do código, utilize objetos intermediários.

Agora vamos juntar todos os passos realizados até agora utilizando pipes.

In [22]:
ocean <- read_csv("https://psyteachr.github.io/msc-data-skills/data/personality.csv") %>%
  gather("question", "score", Op1:Ex9) %>%
  separate(question, c("domain", "qnumber"), sep = 2) %>%
  unite("domain_n", domain, qnumber, sep = "_Q") %>%
  spread(domain_n, score)

Parsed with column specification:
cols(
  .default = col_double(),
  date = col_date(format = "")
)

See spec(...) for full column specifications.



In [23]:
ocean

user_id,date,Ag_Q1,Ag_Q2,Ag_Q3,Ag_Q4,Ag_Q5,Ag_Q6,Ag_Q7,Co_Q1,⋯,Ne_Q6,Ne_Q7,Ne_Q8,Op_Q1,Op_Q2,Op_Q3,Op_Q4,Op_Q5,Op_Q6,Op_Q7
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2006-03-23,2,1,1,1,3,NA,3,3,⋯,1,NA,2,3,6,2,3,6,0,0
1,2006-02-08,0,6,0,0,6,6,0,0,⋯,6,0,0,6,0,6,0,6,6,6
2,2005-10-24,0,6,1,1,5,5,1,0,⋯,5,1,1,6,0,5,1,5,5,5
5,2005-12-07,4,0,1,4,0,2,1,3,⋯,1,2,1,6,4,5,6,2,1,5
8,2006-07-27,6,5,0,6,2,3,1,5,⋯,0,0,1,6,6,5,6,5,6,5
108,2006-02-28,5,4,4,5,5,4,3,4,⋯,1,2,1,3,4,4,3,4,4,4
233,2006-11-08,5,5,4,5,6,5,3,3,⋯,3,4,5,3,4,3,3,3,6,6
298,2005-11-08,4,3,4,6,2,6,5,4,⋯,4,4,4,6,0,2,0,2,0,2
426,2006-09-05,2,4,3,6,2,1,0,5,⋯,0,0,0,6,0,4,6,6,0,1


## 4.7. Um exemplo mais complexo

### 4.7.1. Carregando os dados

Carregue dados relativos à mortalidade infantil a partir do arquivo [imfort.csv](https://psyteachr.github.io/msc-data-skills/data/infmort.csv) no diretório `data`.

In [24]:
dir.create('data', showWarnings = FALSE)

infmort <- read_csv("data/infmort.csv")

Parsed with column specification:
cols(
  Country = col_character(),
  Year = col_double(),
  `Infant mortality rate (probability of dying between birth and age 1 per 1000 live births)` = col_character()
)



In [25]:
glimpse(infmort)

Observations: 5,044
Variables: 3
$ Country                                                                                     <chr> …
$ Year                                                                                        <dbl> …
$ `Infant mortality rate (probability of dying between birth and age 1 per 1000 live births)` <chr> …


Carregue dados de mortalidade maternal do arquivo Excel [matmort.xls](https://psyteachr.github.io/msc-data-skills/data/matmort.xls) no diretório `data`.

In [27]:
matmort <- read_xls("data/matmort.xls")
glimpse(matmort)

Observations: 181
Variables: 4
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ `1990`  <chr> "1 340 [ 878 - 1 950]", "71 [ 58 -  88]", "216 [ 141 -  327]"…
$ `2000`  <chr> "1 100 [ 745 - 1 570]", "43 [ 33 -  56]", "170 [ 118 -  241]"…
$ `2015`  <chr> "396 [ 253 -  620]", "29 [ 16 -  46]", "140 [ 82 -  244]", "4…


Carregue os dados dos códigos de país em [all.csv](https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv)

In [28]:
ccodes <- read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

Parsed with column specification:
cols(
  name = col_character(),
  `alpha-2` = col_character(),
  `alpha-3` = col_character(),
  `country-code` = col_character(),
  `iso_3166-2` = col_character(),
  region = col_character(),
  `sub-region` = col_character(),
  `intermediate-region` = col_character(),
  `region-code` = col_character(),
  `sub-region-code` = col_character(),
  `intermediate-region-code` = col_character()
)



In [29]:
glimpse(ccodes)

Observations: 249
Variables: 11
$ name                       <chr> "Afghanistan", "Åland Islands", "Albania",…
$ `alpha-2`                  <chr> "AF", "AX", "AL", "DZ", "AS", "AD", "AO", …
$ `alpha-3`                  <chr> "AFG", "ALA", "ALB", "DZA", "ASM", "AND", …
$ `country-code`             <chr> "004", "248", "008", "012", "016", "020", …
$ `iso_3166-2`               <chr> "ISO 3166-2:AF", "ISO 3166-2:AX", "ISO 316…
$ region                     <chr> "Asia", "Europe", "Europe", "Africa", "Oce…
$ `sub-region`               <chr> "Southern Asia", "Northern Europe", "South…
$ `intermediate-region`      <chr> NA, NA, NA, NA, NA, NA, "Middle Africa", "…
$ `region-code`              <chr> "142", "150", "150", "002", "009", "150", …
$ `sub-region-code`          <chr> "034", "154", "039", "015", "061", "039", …
$ `intermediate-region-code` <chr> NA, NA, NA, NA, NA, NA, "017", "029", NA, …


### 4.7.2. Largo para longo

`matmort` é no formato largo, com uma coluna para cada ano. Vamos rotacionar a tabela, alterando-a para o formato longo, com uma linha para cada País/Ano.

O exemplo é complicado porque os nomes das colunas reunidas são números. Se os nomes das colunas estão fora do padrão (possuem espaços, começam com números ou possuem caracteres especiais), é possível utilizar crase (\`) como no exemplo a seguir.

In [30]:
matmort_long <- matmort %>%
  gather("Year", "stats", `1990`:`2015`)

glimpse(matmort_long)

Observations: 543
Variables: 3
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ Year    <chr> "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990…
$ stats   <chr> "1 340 [ 878 - 1 950]", "71 [ 58 -  88]", "216 [ 141 -  327]"…


### 4.7.3. Um pedaço dos dados em cada coluna

Os dados na coluna `stats` estão num formato pouco comum, com algum tipo de intervalo de confiança wntre chaves e um monte de espaços extra. Não precisamos dos espaços, então vamos removê-los utilizando a função `mutate`.

A função `separate` vai separar seus dados em qualquer coisa que não seja um número ou uma letra, então tente essa abordagem sem especificar o parâmetro `sep`. O parâmetro `into` é uma lista com o nome das novas colunas.

In [31]:
matmort_split <- matmort_long %>%
  mutate(stats = gsub(" ", "", stats)) %>%
  separate(stats, c("rate", "ci_low", "ci_hi"))

Warning message:
“Expected 3 pieces. Additional pieces discarded in 543 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


In [32]:
glimpse(matmort_split)

Observations: 543
Variables: 5
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ Year    <chr> "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990…
$ rate    <chr> "1340", "71", "216", "1160", "72", "58", "8", "8", "64", "46"…
$ ci_low  <chr> "878", "58", "141", "627", "64", "51", "7", "7", "56", "34", …
$ ci_hi   <chr> "1950", "88", "327", "2020", "80", "65", "9", "10", "74", "61…


#### 4.7.3.1. Trate as colunas a mais com `extra`

In [33]:
matmort_split <- matmort_long %>%
  mutate(stats = gsub(" ", "", stats)) %>%
  separate(stats, c("rate", "ci_low", "ci_hi"), extra = "drop")

glimpse(matmort_split)

Observations: 543
Variables: 5
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ Year    <chr> "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990…
$ rate    <chr> "1340", "71", "216", "1160", "72", "58", "8", "8", "64", "46"…
$ ci_low  <chr> "878", "58", "141", "627", "64", "51", "7", "7", "56", "34", …
$ ci_hi   <chr> "1950", "88", "327", "2020", "80", "65", "9", "10", "74", "61…


#### 4.7.3.2. Ajuste os delimitadores com `sep`

Agora siga o mesmo procedimento com `infmort`. Os dados já estão no formato longo, então não é necessário utilizar `gather`. A terceira coluna tem um nome muito longo, então podemos utilizar apenas o número da coluna (3).

In [34]:
infmort_split <- infmort %>%
  separate(3, c("rate", "ci_low", "ci_hi"), extra = "drop")

glimpse(infmort_split)

Observations: 5,044
Variables: 5
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate    <chr> "66", "68", "69", "71", "73", "75", "76", "78", "80", "82", "…
$ ci_low  <chr> "3", "1", "9", "7", "4", "1", "8", "6", "4", "3", "4", "7", "…
$ ci_hi   <chr> "52", "55", "58", "61", "64", "66", "69", "71", "73", "75", "…


É possível ver que a operação não foi bem sucedida. Os dados estão sendo dividos em novas colunas em espaços, chaves e todos os possíveis delimitadores. Desejamos considerar somente espaços, chaves e hífens. Assim, é necessário ajustar manualmente os delimitadores utilizando `sep`. Também é mais fácil visualizar uma função com muitos argumentos se colocamos um parâmetro por linha.

In [35]:
infmort_split <- infmort %>%
  separate(
    col = 3, 
    into = c("rate", "ci_low", "ci_hi"), 
    extra = "drop", 
    sep = "(\\[|-|])"
  )

glimpse(infmort_split)

Observations: 5,044
Variables: 5
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate    <chr> "66.3 ", "68.1 ", "69.9 ", "71.7 ", "73.4 ", "75.1 ", "76.8 "…
$ ci_low  <chr> "52.7", "55.7", "58.7", "61.6", "64.4", "66.9", "69.0", "71.2…
$ ci_hi   <chr> "83.9", "83.6", "83.5", "83.7", "84.2", "85.1", "86.1", "87.3…


#### 4.7.3.3. Corrija os tipos de dado com `convert`

Observe os caracteres próximos a `Year`, `rate`, `ci_low` e `ci_hi`. Isso significa que as colunas contém caracteres, e não inteiros ou números. Se quisermos calcular a média, por exemplo, a identificação incorreta dos tipos de dado pode ser problemática. Podemos então adicionar o parâmetro `convert` como `TRUE` para corrigir o formato dos dados.

In [36]:
infmort_split <- infmort %>%
  separate(3, c("rate", "ci_low", "ci_hi"), extra = "drop", sep = "(\\[|-|])", convert = TRUE)

glimpse(infmort_split)

Observations: 5,044
Variables: 5
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate    <dbl> 66.3, 68.1, 69.9, 71.7, 73.4, 75.1, 76.8, 78.6, 80.4, 82.3, 8…
$ ci_low  <dbl> 52.7, 55.7, 58.7, 61.6, 64.4, 66.9, 69.0, 71.2, 73.4, 75.5, 7…
$ ci_hi   <dbl> 83.9, 83.6, 83.5, 83.7, 84.2, 85.1, 86.1, 87.3, 88.9, 90.7, 9…


Vamos repetir o procedimento para `matmort`

In [37]:
matmort_split <- matmort_long %>%
  mutate(stats = gsub(" ", "", stats)) %>%
  separate(stats, c("rate", "ci_low", "ci_hi"), extra = "drop", convert = TRUE)

glimpse(matmort_split)

Observations: 543
Variables: 5
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ Year    <chr> "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990…
$ rate    <int> 1340, 71, 216, 1160, 72, 58, 8, 8, 64, 46, 26, 569, 58, 33, 9…
$ ci_low  <int> 878, 58, 141, 627, 64, 51, 7, 7, 56, 34, 20, 446, 47, 28, 7, …
$ ci_hi   <int> 1950, 88, 327, 2020, 80, 65, 9, 10, 74, 61, 33, 715, 72, 38, …


### 4.7.4. Resumindo tudo em um único passo

É possível encadear todos os passos, já que os dataframes intermediários não são necessários.

In [38]:
infmort <- read_csv("data/infmort.csv") %>%
  separate(
    3, 
    c("rate", "ci_low", "ci_hi"), 
    extra = "drop", 
    sep = "(\\[|-|])", 
    convert = TRUE
  )

Parsed with column specification:
cols(
  Country = col_character(),
  Year = col_double(),
  `Infant mortality rate (probability of dying between birth and age 1 per 1000 live births)` = col_character()
)



In [39]:
matmort <- read_xls("data/matmort.xls") %>%
  gather("Year", "stats", `1990`:`2015`) %>%
  mutate(stats = gsub(" ", "", stats)) %>%
  separate(
    stats, 
    c("rate", "ci_low", "ci_hi"), 
    extra = "drop", 
    convert = TRUE
  )

glimpse(matmort)
glimpse(infmort)

Observations: 543
Variables: 5
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Angola", "Argentina", "…
$ Year    <chr> "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990…
$ rate    <int> 1340, 71, 216, 1160, 72, 58, 8, 8, 64, 46, 26, 569, 58, 33, 9…
$ ci_low  <int> 878, 58, 141, 627, 64, 51, 7, 7, 56, 34, 20, 446, 47, 28, 7, …
$ ci_hi   <int> 1950, 88, 327, 2020, 80, 65, 9, 10, 74, 61, 33, 715, 72, 38, …
Observations: 5,044
Variables: 5
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate    <dbl> 66.3, 68.1, 69.9, 71.7, 73.4, 75.1, 76.8, 78.6, 80.4, 82.3, 8…
$ ci_low  <dbl> 52.7, 55.7, 58.7, 61.6, 64.4, 66.9, 69.0, 71.2, 73.4, 75.5, 7…
$ ci_hi   <dbl> 83.9, 83.6, 83.5, 83.7, 84.2, 85.1, 86.1, 87.3, 88.9, 90.7, 9…


### 4.7.5. Colunas por ano

Descubra a mortalidade infantil por ano.

In [40]:
infmort_wide <- infmort %>%
  spread(Year, rate)

glimpse(infmort_wide)

Observations: 4,934
Variables: 29
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ ci_low  <dbl> 52.7, 55.7, 58.7, 61.6, 64.4, 66.9, 69.0, 71.2, 73.4, 75.5, 7…
$ ci_hi   <dbl> 83.9, 83.6, 83.5, 83.7, 84.2, 85.1, 86.1, 87.3, 88.9, 90.7, 9…
$ `1990`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1991`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1992`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1993`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1994`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1995`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1996`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1997`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1998`  <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ `1999`  <dbl> NA

Perceba que não foi possível obter os dados esperados. Isso acontece porque a função `spread()` é aplicada em todas as colunas remanescentes. Assim, como `Afghanistan` com `ci_low = 52.7` é tratado como uma  observação diferente de `ci_low = 55.7`. É possível corrigir o problema juntando novamente as colunas `rate`, `ci_low` e `ci_hi`.

### 4.7.6. Juntando colunas

Junte a taxa e o intervalo de confiança em uma única coluna

In [41]:
infmort_united <- infmort %>%
  unite(rate_ci, rate, ci_low, ci_hi)

glimpse(infmort_united)

Observations: 5,044
Variables: 3
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate_ci <chr> "66.3_52.7_83.9", "68.1_55.7_83.6", "69.9_58.7_83.5", "71.7_6…


#### 4.7.6.1. Controle a separação de nomes com `sep`

`unite()` separa nomes nomes reunidos com um _underscore_ por padrão. Ajuste `sep` se quiser alterar esse comportamento.

In [42]:
infmort_united <- infmort %>%
  unite(rate_ci, rate, ci_low, ci_hi, sep = ", ")

glimpse(infmort_united)

Observations: 5,044
Variables: 3
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate_ci <chr> "66.3, 52.7, 83.9", "68.1, 55.7, 83.6", "69.9, 58.7, 83.5", "…


Para alterar o formato de volta para `rate [ci_low - ci_hi]` utilize `mutate` e `paste`.

In [43]:
infmort_united <- infmort %>%
  mutate(rate_ci = paste0(rate, " [", ci_low, " - ", ci_hi, "]"))

glimpse(infmort_united)

Observations: 5,044
Variables: 6
$ Country <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "…
$ Year    <dbl> 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2…
$ rate    <dbl> 66.3, 68.1, 69.9, 71.7, 73.4, 75.1, 76.8, 78.6, 80.4, 82.3, 8…
$ ci_low  <dbl> 52.7, 55.7, 58.7, 61.6, 64.4, 66.9, 69.0, 71.2, 73.4, 75.5, 7…
$ ci_hi   <dbl> 83.9, 83.6, 83.5, 83.7, 84.2, 85.1, 86.1, 87.3, 88.9, 90.7, 9…
$ rate_ci <chr> "66.3 [52.7 - 83.9]", "68.1 [55.7 - 83.6]", "69.9 [58.7 - 83.…


Vamos tentar separar por ano utilizando `spread()` outra vez. Agora vamos utilizar `rate:ci_hi` ao invés de `rate, ci_low, ci_hi`.

In [44]:
infmort_wide <- infmort %>%
  unite(rate_ci, rate:ci_hi, sep = ", ") %>%
  spread(Year, rate_ci)

glimpse(infmort_wide)

Observations: 194
Variables: 27
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "An…
$ `1990`  <chr> "122.5, 111.6, 135.5", "35.1, 31.3, 39.2", "39.7, 37.1, 42.3"…
$ `1991`  <chr> "118.3, 108, 129.9", "33.7, 30.2, 37.6", "38.8, 36.1, 41.6", …
$ `1992`  <chr> "114.4, 104.6, 125.2", "32.5, 29.2, 36.2", "38.1, 35.4, 41", …
$ `1993`  <chr> "110.9, 101.4, 120.9", "31.4, 28.2, 34.9", "37.5, 34.9, 40.3"…
$ `1994`  <chr> "107.7, 98.6, 117.2", "30.3, 27.1, 33.8", "36.9, 34.6, 39.4",…
$ `1995`  <chr> "105, 96.2, 114.1", "29.1, 26, 32.7", "36.3, 34.2, 38.4", "5.…
$ `1996`  <chr> "102.7, 94.5, 111.3", "27.9, 24.8, 31.5", "35.7, 34, 37.4", "…
$ `1997`  <chr> "100.7, 92.9, 109.1", "26.8, 23.6, 30.4", "35.1, 33.8, 36.5",…
$ `1998`  <chr> "98.9, 91.4, 107.2", "25.5, 22.4, 29.2", "34.7, 33.7, 35.8", …
$ `1999`  <chr> "97.2, 89.9, 105.4", "24.4, 21.2, 28.1", "34.4, 33.5, 35.2", …
$ `2000`  <chr> "95.4, 88.2, 103.6", "23.2, 20, 27", "33.9, 33.2, 34.7", "3.9…
$ `2001`  <chr> "93.

Veja a documentação para entender as operações.

In [46]:
?tidyr::unite

In [52]:
infmort_wide <- infmort %>%
  unite(rate_ci, rate:ci_hi, sep = ", ") %>%
  spread(Year, rate_ci)

glimpse(infmort_wide)

Observations: 194
Variables: 27
$ Country <chr> "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "An…
$ `1990`  <chr> "122.5, 111.6, 135.5", "35.1, 31.3, 39.2", "39.7, 37.1, 42.3"…
$ `1991`  <chr> "118.3, 108, 129.9", "33.7, 30.2, 37.6", "38.8, 36.1, 41.6", …
$ `1992`  <chr> "114.4, 104.6, 125.2", "32.5, 29.2, 36.2", "38.1, 35.4, 41", …
$ `1993`  <chr> "110.9, 101.4, 120.9", "31.4, 28.2, 34.9", "37.5, 34.9, 40.3"…
$ `1994`  <chr> "107.7, 98.6, 117.2", "30.3, 27.1, 33.8", "36.9, 34.6, 39.4",…
$ `1995`  <chr> "105, 96.2, 114.1", "29.1, 26, 32.7", "36.3, 34.2, 38.4", "5.…
$ `1996`  <chr> "102.7, 94.5, 111.3", "27.9, 24.8, 31.5", "35.7, 34, 37.4", "…
$ `1997`  <chr> "100.7, 92.9, 109.1", "26.8, 23.6, 30.4", "35.1, 33.8, 36.5",…
$ `1998`  <chr> "98.9, 91.4, 107.2", "25.5, 22.4, 29.2", "34.7, 33.7, 35.8", …
$ `1999`  <chr> "97.2, 89.9, 105.4", "24.4, 21.2, 28.1", "34.4, 33.5, 35.2", …
$ `2000`  <chr> "95.4, 88.2, 103.6", "23.2, 20, 27", "33.9, 33.2, 34.7", "3.9…
$ `2001`  <chr> "93.